In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('data/smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [15]:
df['message']

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [14]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X = df['message']

In [20]:
y = df['label']

In [22]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [23]:
y

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: label, Length: 5572, dtype: object

In [35]:
X.shape

(5572,)

In [36]:
y.shape

(5572,)

In [46]:
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [47]:
len(X_train)

3733

In [48]:
X_test.shape

(1839,)

In [49]:
len(X_test)

1839

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
count_vect = CountVectorizer()

In [53]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

## Fit vectorizer to the data ( build a vacab, count the number of words)

In [54]:
count_vect.fit(X_train)

CountVectorizer()

## Transform to Text message to vector

In [64]:
x_train_counts = count_vect.transform(X_train)

In [65]:
x_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [66]:
from sklearn.feature_extraction.text import TfidfTransformer

In [67]:
tfidf = TfidfTransformer()

In [69]:
X_train_tfidf = tfidf.fit_transform(x_train_counts)

In [71]:
X_train_tfidf

<3733x7082 sparse matrix of type '<class 'numpy.float64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [73]:
X_train_tfidf.shape

(3733, 7082)

### Instead of count vectorizer and TFIDF transformer we can directly use the TfidfVectorizer

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [76]:
vectorizer = TfidfVectorizer()

In [79]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [80]:
X_train_tfidf

<3733x7082 sparse matrix of type '<class 'numpy.float64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [81]:
from sklearn.svm import LinearSVC

In [82]:
clf = LinearSVC()

In [84]:
clf.fit(X_train_tfidf, y_train)

LinearSVC()

## To avoid repeat the same activity on Test data, we can do the Pipeline

In [85]:
from sklearn.pipeline import Pipeline

In [87]:
pipeline = Pipeline([('vector', TfidfVectorizer()),('classifier', LinearSVC())])

In [88]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vector', TfidfVectorizer()), ('classifier', LinearSVC())])

In [89]:
predictions = pipeline.predict(X_test)

In [97]:
from sklearn.metrics import confusion_matrix, classification_report

In [99]:
print(confusion_matrix(y_test, predictions))

[[1586    7]
 [  12  234]]


In [102]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [103]:
from sklearn import metrics

In [105]:
metrics.accuracy_score(y_test, predictions)

0.989668297988037

In [106]:
pipeline.predict(['How are you'])

array(['ham'], dtype=object)

In [107]:
pipeline.predict(['You won 500$'])

array(['spam'], dtype=object)